# Tilkallinger basert på forskjellige perioder


In [2]:
# Vi henter alle dataene på nytt for å holde orden i koden 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


fil_navn = 'DIN_FILNAVN.xlsx'
df = pd.read_excel(fil_navn)

In [ ]:
# Ser på de første 5 radene for å sikre at dataen er med slik som de er
df.head(n=5)

In [ ]:
# Ser på de siste 5 radene for å sikre at dataen er med slik som de er
df.tail(n=5)

In [5]:
df['Pl. ferdig'] = pd.to_datetime(df['Pl. ferdig'])  # Konverter "Pl. ferdig"-kolonnen til en datetime
monthly_calls = df.groupby(df['Pl. ferdig'].dt.to_period("M")).size() # Grupper dataene per måned og tell antall vakttilkallinger

## Visualisere antall tilkallinger for hver mnd 

In [ ]:
average_calls = monthly_calls.mean()

plt.figure(figsize=(15, 6))
monthly_calls.plot(kind='bar', alpha=0.75)  # Reduserer opaciteten litt for å se gjennomsnittslinjen tydeligere
plt.axhline(y=average_calls, color='r', linestyle='--', label=f'Gjennomsnitt: {average_calls:.2f}')
plt.title("Antall vakttilkallinger per måned")
plt.xlabel("Måned")
plt.ylabel("Antall vakttilkallinger")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

### Hvilket jobområdet tilhører alle dataene tilkallingene? 



In [ ]:
#gruppere etter Jobbområde
grouped_df = df.groupby('Jobbomr.').size()
grouped_df.name ="Count"

print((grouped_df))

dataFrameJobbOm = grouped_df.reset_index()
dataFrameJobbOm

plt.figure(figsize=(16, 8))

# Create a bar plot
plt.subplot(1, 2, 2)
# plt.figure(figsize=(10, 6))
plt.bar(dataFrameJobbOm['Jobbomr.'], dataFrameJobbOm['Count'], color=['blue', 'green'])
plt.xlabel('Jobbomr.')
plt.ylabel('Antall')
plt.title('Antall jobber fordelt på Jobb område')
# plt.show()

# Create a pie chart
plt.subplot(1, 2, 1)
# plt.figure(figsize=(8, 8))
plt.pie(dataFrameJobbOm['Count'], labels=dataFrameJobbOm['Jobbomr.'], autopct='%1.1f%%', startangle=50,)
# plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribusjon av Jobb område')
plt.show()


# Analysere "Verstingene" 
Nå skal vi analysere hvert objekt alene og finne ut mere informasjon i datasettet. 

### Vi finner først ut hvor mange objekter vi har.  

In [ ]:
# Finne ut alle kolonne navnene og srive ut  

column_name = list(df.columns)
print(column_name)

# antal unike objekter

unique_objekts = df['Objektnr'].unique()
unique_objekts_len = len(list(unique_objekts))

print("Antall unike objekter: ", unique_objekts_len) # vi har totalt 884 forskjellige objekter
# print("Unike ojekter navn:\n", unique_objekts) # Du kan skrive ut alle navnene her 


# Filtrering og forberedelser av data

Vårt data sett er for 3 års periode. og vi kan se i datasettet vårt at det er mange objekter som kun har hatt en vakttilkalling. Noen viktige punkter:

- Vi må bestemme oss for objekter objekter vi skal analysere
- Hvordan skal vi velge hvem vi skal analysere? 


In [ ]:
# Først grupperer vi objektene etter antall tilkallinger
grouped_df = df.groupby('Objektnr').size()
grouped_df.name = "Antall"

# Konvertere dataserie til data frame
grouped_df = grouped_df.reset_index()

# VI finner den verste og begynner med den 

# Printe ut antall objekter med antall tilkallinger 
print("Objektnr med antall tilkallinger: ")
print(grouped_df)

# vi sortere etter de med størst antall 
sorted_df = grouped_df.sort_values(by='Antall', ascending=False)

print("Objektnr med antall tilkallinger sortert [største -> minst]: ")
print(sorted_df)

# printer de 20 "verste"
sorted_df.head(n=20)



### Vi begynner å analysere de 20 objektene som har hatt mest vakttilkallinger gjennom alle tre årene



### Andel av totalen 

Vi kan først se hvor mye de "20 verstingene" utgjør i forhold til resten a datasettet. Vi kan visualisere ved hjelp av kakediagram. 


In [ ]:
# de verste 20 
worst_20 = sorted_df.head(n=20)

# resetter indeksene 
worst_20.reset_index(drop=True, inplace=True)

antall_tilkallinger = df['Objektnr'].count()
antall_tilkallinger_worst_20 = sum(worst_20['Antall'])
antall_tilkallinger_others = antall_tilkallinger - antall_tilkallinger_worst_20

# Data for det andre kakediagrammet
total_unike_objekter = df['Objektnr'].nunique()

worst_20_objekter = 20
andre_objekter = total_unike_objekter - worst_20_objekter

labels2 = ['Verste 20 objekter', f'Andre {andre_objekter} objekter']
sizes2 = [worst_20_objekter, andre_objekter]
colors2 = ['blue', 'orange']
explode2 = (0.1, 0)

# Data for det første kakediagrammet
labels1 = ['Verste 20 objekter', f'Andre {andre_objekter} objekter']
sizes1 = [antall_tilkallinger_worst_20, antall_tilkallinger_others]
colors1 = ['red', 'green']
explode1 = (0.1, 0)

# Lage kakediagrammene
plt.figure(figsize=(16, 8))

# Første kakediagram
plt.subplot(1, 2, 2)
plt.pie(sizes1, explode=explode1, labels=labels1, colors=colors1, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Fordeling av vakttilkallinger")

# Andre kakediagram
plt.subplot(1, 2, 1)
plt.pie(sizes2, explode=explode2, labels=labels2, colors=colors2, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title("Fordeling av objekter")

plt.show()

# Alle objektnr
worst_20_objektnr = worst_20['Objektnr']


filtered_df = df[df['Objektnr'].isin(worst_20_objektnr)]

filtered_df

### Visualisere antall tilkallinger for hver mnd 

In [ ]:

start_date = filtered_df['Pl. ferdig'].min().to_period('M')
end_date = filtered_df['Pl. ferdig'].max().to_period('M')
all_months = pd.period_range(start_date, end_date, freq='M')

monthly_calls_worst_20 = filtered_df.groupby(filtered_df['Pl. ferdig'].dt.to_period("M")).size()

monthly_calls_worst_20 = monthly_calls_worst_20.reindex(all_months, fill_value=0)


average_calls_worst_20 = monthly_calls_worst_20.mean()

plt.figure(figsize=(15, 6))
monthly_calls_worst_20.plot(kind='bar', alpha=0.75)  # Reduserer opaciteten litt for å se gjennomsnittslinjen tydeligere
plt.axhline(y=average_calls_worst_20, color='r', linestyle='--', label=f'Gjennomsnitt: {average_calls_worst_20:.2f}')
plt.title("Antall vakttilkallinger per måned")
plt.xlabel("Måned")
plt.ylabel("Antall vakttilkallinger")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

#### Visualisere hvert objekt's vakttilkallinger for alle tre årene for hver mnd 

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(10,40))

# Flatten the array of axis subplots, so we can easily loop over it
axs = axs.flatten()

# Initialize variable to keep track of the maximum y-value across all plots
max_y_value = 0

# Create a mapping from Objektnr to Objektnavn
objektnr_to_name = dict(zip(filtered_df['Objektnr'], filtered_df['Objektnavn']))

# First pass to find the maximum y-value
for objektnr in worst_20['Objektnr']:
    temp_df = filtered_df[filtered_df['Objektnr'] == objektnr]
    monthly_calls = temp_df.groupby(temp_df['Pl. ferdig'].dt.to_period("M")).size()
    max_y_value = max(max_y_value, monthly_calls.max())

# Second pass to plot the data
for i, objektnr in enumerate(worst_20['Objektnr']):
    temp_df = filtered_df[filtered_df['Objektnr'] == objektnr]
    monthly_calls = temp_df.groupby(temp_df['Pl. ferdig'].dt.to_period("M")).size()
    
    # Create a complete index that includes all months from the earliest to the latest date
    start_date = temp_df['Pl. ferdig'].min().to_period('M')
    end_date = temp_df['Pl. ferdig'].max().to_period('M')
    all_months = pd.period_range(start_date, end_date, freq='M')
    
    # Reindex your series
    monthly_calls = monthly_calls.reindex(all_months, fill_value=0)
    
    average_calls = monthly_calls.mean()

    objectName = objektnr_to_name.get(objektnr, "Unknown")
    
    axs[i].bar(monthly_calls.index.astype(str), monthly_calls.values, alpha=0.75)
    axs[i].axhline(y=average_calls, color='r', linestyle='--', label=f'Avg: {average_calls:.2f}')
    axs[i].set_title(f"{objektnr} - {objectName}")
    axs[i].set_xticklabels(monthly_calls.index.astype(str), rotation=45)
    axs[i].set_ylim([0, max_y_value])
    axs[i].legend()

# Remove any unused subplots
for i in range(len(worst_20['Objektnr']), len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.show()


In [ ]:
def moving_average(data, window_size):
    """Calculate a simple moving average."""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

# Create a mapping from Objektnr to Objektnavn
objektnr_to_name = dict(zip(filtered_df['Objektnr'], filtered_df['Objektnavn']))

fig, axs = plt.subplots(10, 2, figsize=(10, 40))
axs = axs.flatten()

max_y_value = 0

# First pass to find the maximum y-value
for objektnr in worst_20['Objektnr']:
    temp_df = filtered_df[filtered_df['Objektnr'] == objektnr]
    monthly_calls = temp_df.groupby(temp_df['Pl. ferdig'].dt.to_period("M")).size()
    max_y_value = max(max_y_value, monthly_calls.max())

# Second pass to plot the data
for i, objektnr in enumerate(worst_20['Objektnr']):
    temp_df = filtered_df[filtered_df['Objektnr'] == objektnr]
    monthly_calls = temp_df.groupby(temp_df['Pl. ferdig'].dt.to_period("M")).size()
    average_calls = monthly_calls.mean()
    
    # Calculate the moving average with a window size of 3
    smoothed_calls = moving_average(monthly_calls, 3)
    
    objectName = objektnr_to_name.get(objektnr, "Unknown")
    
    axs[i].bar(monthly_calls.index.astype(str), monthly_calls.values, alpha=0.4, label='Monthly Calls')
    axs[i].plot(monthly_calls.index.astype(str)[:-2], smoothed_calls, color='r', linestyle='-', label='Smoothed Curve')
    axs[i].set_title(f"{objektnr} - {objectName}")
    axs[i].set_xticklabels(monthly_calls.index.astype(str), rotation=45)
    axs[i].set_ylim([0, max_y_value])
    axs[i].legend()

# Remove any unused subplots
for i in range(len(worst_20['Objektnr']), len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.show()
